In [16]:
import pandas as pd
import seaborn as sns
import numpy as np

In [17]:
#bring in combined pickle file all years 2015-2017

df_payments_combined = pd.read_pickle('../data/df_payments_combined.pkl')

- Chart showing payments (average_medicare_allowed_amount) over time
  -Two charts, based on Entity (I, O) for Individual and for Organization
- Chart showing counts over time (count of Beneficiaries and count of Services)

# Creating New Column: payment_type

- Entity I, POS F = Doctor Only
- Entity O, POS F = Facility Only
- Entity I, POS O = Doctor & Facility
- Entity O, POS O = Doctor & Facility

In [18]:
conditions = [
    (df_payments_combined['place_of_service'] == 'O'),
    (df_payments_combined['entity_type_of_the_provider'] == 'I') & (df_payments_combined['place_of_service'] == 'F'),
    (df_payments_combined['entity_type_of_the_provider'] == 'O') & (df_payments_combined['place_of_service'] == 'F')]

choices = ['Doctor & Facility','Doctor Only', 'Facility Only']

In [19]:
df_payments_combined['payment_type'] = np.select(conditions, choices, default='unknown')

In [23]:
df_payments_combined.tail()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year,payment_type
29060225,1992999825,DESCHENES,I,SEATTLE,981012756,WA,Otolaryngology,O,99214,Established patient office or other outpatient...,248.0,175.0,248.0,116.86,2017,Doctor & Facility
29060226,1992999874,JOFFE,I,MECHANICSVILLE,231161844,VA,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",41.0,41.0,41.0,203.40,2017,Doctor Only
29060227,1992999874,JOFFE,I,MECHANICSVILLE,231161844,VA,Internal Medicine,F,99232,"Subsequent hospital inpatient care, typically ...",101.0,57.0,101.0,72.48,2017,Doctor Only
29060228,1992999874,JOFFE,I,MECHANICSVILLE,231161844,VA,Internal Medicine,F,99233,"Subsequent hospital inpatient care, typically ...",102.0,55.0,102.0,104.76,2017,Doctor Only
29060229,1992999874,JOFFE,I,MECHANICSVILLE,231161844,VA,Internal Medicine,F,99239,"Hospital discharge day management, more than 3...",49.0,49.0,49.0,107.98,2017,Doctor Only


In [24]:
df_payments_combined['payment_type'].value_counts()

Doctor & Facility    17733816
Doctor Only          11048039
Facility Only          278374
unknown                     1
Name: payment_type, dtype: int64

# Groupby NPI/HCPCS/POS/Entity; POS/Entity is now payment_type

In [90]:
%%time
df_avg_pmt = df_payments_combined.groupby(['year',
                                           'payment_type',
                                           'hcpcs_code']).average_medicare_allowed_amount.mean().to_frame().reset_index()

CPU times: user 6 s, sys: 3.38 s, total: 9.38 s
Wall time: 10.6 s


In [91]:
df_avg_pmt

,year,payment_type,hcpcs_code,average_medicare_allowed_amount
0,2015,Doctor & Facility,00103,255.076212
1,2015,Doctor & Facility,00104,111.614526
2,2015,Doctor & Facility,00140,139.047955
3,2015,Doctor & Facility,00142,102.570003
4,2015,Doctor & Facility,00160,211.812813
...,...,...,...,...
28874,2017,Facility Only,Q4172,117.524101
28875,2017,Facility Only,Q9968,3.404030
28876,2017,Facility Only,R0070,180.445074
28877,2017,Facility Only,R0075,80.649999


# Measure how payments and counts change over time
“Counts” = count of Beneficiaries and count of Services

Mean of number_of_distinct_medicare_beneficiary_per_day_services PER hcpcs PER year

In [92]:
df_avg_services_day = df_payments_combined.groupby(['year',
                                                    'hcpcs_code']).number_of_distinct_medicare_beneficiary_per_day_services.mean().to_frame().reset_index()

In [93]:
df_avg_services_day

,year,hcpcs_code,number_of_distinct_medicare_beneficiary_per_day_services
0,2015,00100,15.000000
1,2015,00102,26.000000
2,2015,00103,29.192544
3,2015,00104,48.568561
4,2015,00120,22.018182
...,...,...,...
18048,2017,Q9989,16.000000
18049,2017,R0070,2870.982456
18050,2017,R0075,1917.967213
18051,2017,V2632,89.000000


Median of number_of_distinct_medicare_beneficiary_per_day_services PER hcpcs PER year

In [94]:
df_med_services_day = df_payments_combined.groupby(['year',
                                                    'hcpcs_code']).number_of_distinct_medicare_beneficiary_per_day_services.median().to_frame().reset_index()

In [95]:
df_med_services_day

,year,hcpcs_code,number_of_distinct_medicare_beneficiary_per_day_services
0,2015,00100,13.5
1,2015,00102,18.0
2,2015,00103,19.0
3,2015,00104,26.0
4,2015,00120,15.0
...,...,...,...
18048,2017,Q9989,16.0
18049,2017,R0070,676.0
18050,2017,R0075,302.5
18051,2017,V2632,89.0


May be merging dataframes, TBD - merged, with separate row for each NPI, with separate columns for each year’s payments, etc.?

In [86]:
#df_payments_merge1 = pd.merge(df_payments_2015, df_payments_2016, on = 'national_provider_identifier', how = 'outer')
#df_payments_combmerge = pd.merge(df_payments_merge1, df_payments_2017, on = 'national_provider_identifier', how='outer')